# DATA LOADING & CLEANING

In [538]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [539]:
import os
os.chdir('/Users/ltran/Documents/TrueData29/CPC_ML_tutorial/')

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import random

# Set data directory
data_dir = '/Users/ltran/Documents/Data/'

# Set working directory
root_dir = '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

In [540]:
! rm -r '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/models/test/'
! mkdir '/Users/ltran/Documents/TrueData0104/CPC_ML_tutorial/models/test/'

# ANATOMY

### RDB Data

In [541]:
# Extract scanner 
instru = pd.read_csv(os.path.join(data_dir, 'TSA_cohort_dcm_info_anonymized.csv'), sep = '\t')

# Load participants infos (age, sex, diagnosis, scores...)
new = pd.read_csv(os.path.join(data_dir, 'participants.tsv'), sep = "\t")
new = new.drop(index = new.loc[new['ASD'] == '?'].index)

new = new.merge(instru.rename(columns = {'SubCode BIDS' : 'participant_id'}))

# Keep only T1.5 scanner in RDB data
new = new[new.participant_id.isin(instru.loc[instru.magnetic_field_strength == '1_5']['SubCode BIDS'])]

# Load cortical thickness of right and left hemisphere
lh_RDB = pd.read_csv(os.path.join(data_dir,'fs-6.0.0-tfe.lh.thickness.csv'), sep = '\t')
rh_RDB = pd.read_csv(os.path.join(data_dir,'fs-6.0.0-tfe.rh.thickness.csv'), sep = '\t')

# Convert BIDS ID to int format
l_part_id = []
for i in lh_RDB['lh.aparc.thickness']:
    
    l_part_id.append(int(i.split('-')[1].split('_')[0].lstrip('0')))
lh_RDB['participant_id'] = l_part_id

# Merge right and left hemisphere
df_RDB = lh_RDB.join(rh_RDB.drop(columns = ['BrainSegVolNotVent', 'eTIV'])).merge(new, on = 'participant_id', how = 'inner')


# Format 'sex' as boolean
df_RDB['sex'] = df_RDB['sex'].replace({'F' : -1, 'M' : 1})
#Drop duplicates
df_RDB = df_RDB.drop_duplicates(subset = ['participant_id'])

# Load clinical scores
clinic_rdb = pd.read_csv(os.path.join(data_dir, 'MRI_RDB_20190423_anonym_updated.tsv'), sep = '\t')

clinic_rdb['participant_id'] = clinic_rdb['subject_id']

# Match clinical scores names of RDB and EU-AIMS
clinic_rdb = clinic_rdb.rename( columns = {'ADI_R_SOCIAL_TOTAL_A': 't1_adi_social_total',
    'ADI_R_RRB_TOTAL_C' :'t1_adi_rrb_total',
    'RBS-R tot': 't1_rbs_total',
    'SRS': 't1_srs_tscore'})

df_RDB = clinic_rdb.merge(df_RDB, on = ['participant_id', 'ASD']).drop_duplicates(subset = ['participant_id'])

Removing failed QC from RDB data

In [542]:
QC_rdb = pd.read_csv(os.path.join(data_dir, 'QC_trio.csv'))

f_id = []
for i in QC_rdb.loc[QC_rdb.QC == 'F'].BIDS:
    f_id.append(int(i.split('-')[1].strip('0')))

f_id.append(430)

df_RDB = df_RDB.loc[~df_RDB['subject_id'].isin(f_id)]



Separate scanners into two different sites in RDB

In [543]:


df_RDB.loc[(df_RDB['manufacturer_model_name'] == 'INTERA'), 'site'] = 'RDB_INTERA'
df_RDB.loc[(df_RDB['manufacturer_model_name'] == 'INTERA'), 't1_site'] = 9

df_RDB.loc[(df_RDB['manufacturer_model_name'] == 'INGENIA'), 'site'] = 'RDB_INGENIA'
df_RDB.loc[(df_RDB['manufacturer_model_name'] == 'INGENIA'), 't1_site'] = 10

# Diagnosis status for TD relatives = 'Relative'

df_RDB.loc[(df_RDB.Group == 'Relative') & (df_RDB.ASD == 'No'), 'ASD'] = 'Relative'

df_RDB

,subject_id,session_id,Sex,Group,FamilyStatus_x,Control,ASD,acq_time,age_at_scan,machine,...,MRI_code_old,patient_birth_date,patient_sex,patient_weight,acquisition_date,manufacturer,manufacturer_model_name,magnetic_field_strength,site,t1_site
0,3,1,Female,Patient,Proband,No,Yes,2006-02-16T19:18:00,9.4,Intera,...,MRIPasteur-413,19960924,F,20,20060216,PHILIPS_MEDICAL_SYSTEMS,INTERA,1_5,RDB_INTERA,9.0
4,25,1,Male,Patient,Proband,No,Yes,2007-01-22T10:49:05,6.4,Intera,...,MRIPasteur-068,20000906,M,35,20070122,PHILIPS_MEDICAL_SYSTEMS,INTERA,1_5,RDB_INTERA,9.0
8,70,3,Male,Patient,Proband,No,Yes,2014-12-06T10:35:48,11.3,Ingenia,...,MRIPasteur-081,20030901,M,17,20080108,PHILIPS_MEDICAL_SYSTEMS,INTERA,1_5,RDB_INTERA,9.0
26,95,1,Female,Patient,Proband,No,Yes,2008-05-07T10:50:19,14.5,Intera,...,MRIPasteur-128,19931117,F,55,20080507,PHILIPS_MEDICAL_SYSTEMS,INTERA,1_5,RDB_INTERA,9.0
30,96,1,Male,Patient,Proband,No,Yes,2008-05-16T13:45:57,4.9,Intera,...,MRIPasteur-046,20030620,M,22,20080516,PHILIPS_MEDICAL_SYSTEMS,INTERA,1_5,RDB_INTERA,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,1401,1,Male,Patient,Proband,No,Yes,2019-09-16T12:45:28,16.2,Ingenia,...,NaN,20030721,M,40,20190916,PHILIPS_HEALTHCARE,INGENIA,1_5,RDB_INGENIA,10.0
907,1402,1,Male,Patient,Proband,No,Yes,2019-09-19T17:33:19,10.7,Ingenia,...,NaN,20090111,M,25,20190919,PHILIPS_HEALTHCARE,INGENIA,1_5,RDB_INGENIA,10.0
908,1411,1,Male,Patient,Proband,No,Yes,2019-10-28T10:03:08,15.4,Ingenia,...,NaN,20040524,M,50,20191028,PHILIPS_HEALTHCARE,INGENIA,1_5,RDB_INGENIA,10.0
909,1425,1,Female,Patient,Proband,No,Yes,2019-11-08T16:02:26,2.9,Ingenia,...,NaN,20161205,F,14,20191108,PHILIPS_HEALTHCARE,INGENIA,1_5,RDB_INGENIA,10.0


###  EU-AIMS Data

In [544]:
# Load EU-AIMS data
QC = pd.read_csv(os.path.join(data_dir,'QC_LEAP_freesurfer_preproc_info.csv'), sep = ';')

leap = pd.read_csv(os.path.join(data_dir,'LEAP_CorticalMeasuresENIGMA_ThickAvg.csv'), sep = ';')

cov = pd.read_csv(os.path.join(data_dir,'Core_variables_03_09_19_withvalues_internal.tsv'), sep = '\t')

# Merge autism scores with QC file, keep 'include' only
df_cov_aims = cov.merge(QC.loc[QC['final_decision_bin'] == "include"], on = "participant_id", how = "inner")

# ID to int format
l_id  = []
for i in leap['SubjID']:
    l_id.append(int(i.split('_')[1]))

leap['participant_id'] = l_id

# Merge thickness values with infos
df_aims = leap.merge(df_cov_aims, on = 'participant_id', how = 'inner')

### Pooling EU-AIMS et RDB

# Rename columns of EU-AIMS dataframe to merge with RDB
df_aims.columns = df_aims.columns.str.replace('L_','lh_')
df_aims.columns = df_aims.columns.str.replace('_thickavg','_thickness')
df_aims.columns = df_aims.columns.str.replace('R_','rh_')
df_aims = df_aims.rename(columns = {'t1_ageyrs' : 'age_at_scan', 't1_sex' : 'sex', 't1_fsiq' : 'Total_IQ'})

# ASD = group 2 + 4 | TD = group 1 | Drop all rows beloging to group 3
df_aims['ASD'] = df_aims['t1_group'].replace({1 : 'No', 2 : 'Yes', 4 : 'Yes', 3 : np.nan})


# Extract name of the regions
cols_RDB = [col for col in df_RDB.columns if '_thick' in col]

cols_RDB.remove('rh_MeanThickness_thickness')
cols_RDB.remove('lh_MeanThickness_thickness')

# Merge EU-AIMS and RDB dataframes
df_all = df_aims.merge(df_RDB, on = cols_RDB.append(['age_at_scan', 
                                                     'sex', 'site', 
                                                     't1_site',
                                                    't1_adi_social_total',
                                                    't1_adi_rrb_total',
                                                    't1_rbs_total',
                                                    't1_srs_tscore']), how = 'outer')
df_all = df_all.dropna(subset = ['ASD'])
df_all

df_all

# Filter on age = [6, 30]

df_all = df_all.loc[(df_all.age_at_scan < 31) & (df_all.age_at_scan > 5 )]

df_all = df_all.loc[~(df_all.ASD == 'Relative')]

### ABIDE 

ABIDE 1

In [545]:
abide1_lh = pd.read_csv(os.path.join(data_dir, 'ABIDE1/fs-6.0.0-abide.lh.thickness.csv'), sep = '\t')

In [546]:
abide1_rh = pd.read_csv(os.path.join(data_dir, 'ABIDE1/fs-6.0.0-abide.rh.thickness.csv'), sep = '\t')

In [547]:
ABIDE1_thick = abide1_rh.rename(columns= {'rh.aparc.thickness': 'participant_id'}).merge(abide1_lh.rename(columns= {'lh.aparc.thickness': 'participant_id'}), on = 'participant_id')

In [548]:
part_abd1 = pd.read_csv(os.path.join(data_dir, 'ABIDE1/Phenotypic_V1_0b.csv'), sep = ',')

In [549]:
ABIDE1 = part_abd1.rename(columns= {'SUB_ID' : 'participant_id'}).merge(ABIDE1_thick, on = 'participant_id')

In [550]:
ABIDE1.columns = ABIDE1.columns.str.lower()

ABIDE 2

In [551]:
abide2_lh = pd.read_csv(os.path.join(data_dir, 'ABIDE2/fs-6.0.0-abide2.lh.thickness.csv'), sep = '\t')
abide2_rh = pd.read_csv(os.path.join(data_dir, 'ABIDE2/fs-6.0.0-abide2.rh.thickness.csv'), sep = '\t')

In [552]:
ABIDE2_thick = abide2_rh.merge(abide2_lh.rename(columns= {'lh.aparc.thickness': 'rh.aparc.thickness'}), on = 'rh.aparc.thickness')

In [553]:
abide2_part = pd.read_csv(os.path.join(data_dir, 'ABIDE2/participants-merged.tsv'), sep = '\t',
                         encoding= 'unicode_escape')

In [554]:
participant_id = []
for i in ABIDE2_thick['rh.aparc.thickness']:
    participant_id.append(i.split('-')[1].split('_')[0])

ABIDE2_thick['participant_id'] =  participant_id

In [555]:
ABIDE2 = ABIDE2_thick.merge(abide2_part, on = 'participant_id')

In [556]:
ABIDE2 = ABIDE2.drop_duplicates(subset=['participant_id'])

In [557]:
ABIDE2 = ABIDE2.rename(columns={'age_at_scan ': 'age_at_scan'})

In [558]:
l_site = []
for i in ABIDE2.site_id:
    l_site.append(i.replace('ABIDEII-', ''))
ABIDE2['site_id'] = l_site

Merge ABIDE 1 & ABIDE 2 

In [559]:
ABIDE = pd.concat([ABIDE1, ABIDE2], join = 'inner')

In [560]:
ABIDE = ABIDE.rename(columns = {'site_id' : 'site'})

In [561]:
ABIDE['ASD'] = ABIDE['dx_group'].replace({1 : 'No', '2': 'Yes', '1' : 'No', 2 :'Yes'})

In [562]:
ABIDE.sex = ABIDE.sex.replace({'1' : 1, 2 : -1, '2': -1})

In [564]:
ABIDE.to_csv(os.path.join(data_dir, 'Outputs/df_abide.csv'), index=False)

### Join RDB + EU-AIMS + ABIDE

In [535]:
df_export = pd.concat([df_all, ABIDE], join = 'inner')

In [537]:
df_export.to_csv(os.path.join(data_dir, 'Outputs/df_all.csv'), index=False)
